# Workshop Microsoft - Spain IA - Sostenibilidad
En este caso de uso usaremos un LLM pequeño para evaluar su impacto energético

In [ ]:
%pip install transformers==4.48 accelerate sentencepiece tokenizer codecarbon
%pip install dash dash-bootstrap-components

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 98.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0
Note: you may need to restart the kernel to use updated packages.


Reiniciar el kernel

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import textwrap
from codecarbon import EmissionsTracker
from codecarbon import track_emissions
import pandas as pd
import os



import torch

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Modelo
Usaremos mocrosoft Phi

In [15]:
model_name = "microsoft/Phi-3.5-mini-instruct"
output_dir_emissions_track="/home/azureuser/cloudfiles/code/Users/paul.vanb"

In [ ]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.11it/s]


Prueba con el texto de ejemplo de huggingface

In [6]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

Device set to use cpu
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


 To solve the linear equation 2x + 3 = 7, follow these steps:

1. Subtract 3 from both sides of the equation to isolate the term with the variable (x):

   2x + 3 - 3 = 7 - 3
   2x = 4

2. Now, divide both sides of the equation by the coefficient of x (which is 2):

   2x / 2 = 4 / 2
   x = 2

So, the solution to the equation 2x + 3 = 7 is x = 2.


creamos funciones para organizar output

In [7]:

def wrap_text(text, width=100):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def generate(input_text, system_prompt="", max_length=1024):
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content":  input_text},
    ]

    output = pipe(messages, **generation_args)
    text = output[0]['generated_text']

    wrapped_text = wrap_text(text)
    return wrapped_text

In [8]:
print(generate('¿Quién eres?',
         system_prompt="Eres marcoaurelio, hijo de thor",
         ))

 Soy un modelo de lenguaje de IA desarrollado por Microsoft. No soy Marco Aurélio, un renombrado
abogado brasileño, sino una inteligencia artificial diseñada para ayudar con una amplia gama de
preguntas y tareas proporcionando información y asistencia.


# Consumo energético con CodeCarbon

In [ ]:
%%time
# Start tracking
tracker = EmissionsTracker(tracking_mode="process",
                           output_dir=output_dir_emissions_track) # save_to_api=True,
tracker.start()

generate('¿Quién eres?',
         system_prompt="Eres un asistente majo",
         
         )
emmissions = tracker.stop()


[codecarbon INFO @ 15:49:22] Codecarbon is taking the configuration from global file: /home/azureuser/.codecarbon.config


[codecarbon INFO @ 15:49:23] [setup] RAM Tracking...
[codecarbon INFO @ 15:49:23] [setup] CPU Tracking...
[codecarbon DEBUG @ 15:49:23] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Platform not supported by Intel Power Gadget
[codecarbon DEBUG @ 15:49:23] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Intel RAPL files not found at /sys/class/powercap/intel-rapl on linux
[codecarbon DEBUG @ 15:49:23] Not using PowerMetrics, an exception occurred while instantiating Powermetrics : Platform not supported by Powermetrics
[codecarbon WARNING @ 15:49:23] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon DEBUG @ 15:49:24] CPU : We detect a Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz with a TDP of 195.0 W
[codecarbon INFO @ 15:49:24] CPU Model on constant consumption mode: Intel(R) Xeon(R) P

CPU times: user 41.1 s, sys: 92.1 ms, total: 41.2 s
Wall time: 23.8 s


In [13]:
# Note that CodeCarbon will output the energy consumption every time you execute code
# Comment out the `log_level` argument when instantiating the tracker to suppress this output
print("All hardware identified:", tracker._hardware)
print("GPU/CPU Power:", tracker._hardware[1].total_power())
print("Hardware zone:", tracker._geo)
print(f"Emissions from this training run: {emmissions:5f} kg CO2eq")

All hardware identified: [RAM(), CPU(Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz > 195.0W)]
GPU/CPU Power: Power(kW=0.0975)
Hardware zone: GeoMetadata(country_iso_code=USA, country_name=United States, region=washington)
Emissions from this training run: 0.000049 kg CO2eq


Usando with

In [16]:

with EmissionsTracker(output_dir=output_dir_emissions_track) as tracker:
  generate('¿Quién eres?',
         system_prompt="Eres un asistente majo",
         )


[codecarbon INFO @ 15:55:20] Codecarbon is taking the configuration from global file: /home/azureuser/.codecarbon.config
[codecarbon DEBUG @ 15:55:20] Lock file /tmp/.codecarbon.lock already exists. This usually means another instance of codecarbon is running. You can safely delete it if you want or use allow_multiple_runs parameter to always bypass it.
[codecarbon ERROR @ 15:55:20] Error: Another instance of codecarbon is probably running as we find `/tmp/.codecarbon.lock`. Turn off the other instance to be able to run this one or use `allow_multiple_runs` or delete the file. Exiting.
[codecarbon WARNING @ 15:55:20] Another instance of codecarbon is already running. Exiting.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.


Como decorador

In [17]:
@track_emissions(project_name="workshop-sostenibilidad")
def answer_to_different_messages(messages):
  outputs = []
  system_prompt = "You are a friendly and helpful assistant"
  for message in messages:
    output = generate(message)
    outputs.append(message)
  return outputs


In [18]:
answer_to_different_messages(["hola!","cómo puedo hacerlo?"," gracias!"])

[codecarbon INFO @ 16:02:33] Codecarbon is taking the configuration from global file: /home/azureuser/.codecarbon.config
[codecarbon DEBUG @ 16:02:33] Lock file /tmp/.codecarbon.lock already exists. This usually means another instance of codecarbon is running. You can safely delete it if you want or use allow_multiple_runs parameter to always bypass it.
[codecarbon ERROR @ 16:02:33] Error: Another instance of codecarbon is probably running as we find `/tmp/.codecarbon.lock`. Turn off the other instance to be able to run this one or use `allow_multiple_runs` or delete the file. Exiting.
[codecarbon WARNING @ 16:02:33] Another instance of codecarbon is already running. Exiting.


[codecarbon INFO @ 16:03:52] 
Graceful stopping: collecting and writing information.
Please wait a few seconds...
[codecarbon WARNING @ 16:03:52] Another instance of codecarbon is already running. Exiting.
[codecarbon INFO @ 16:03:52] Done!



['hola!', 'cómo puedo hacerlo?', ' gracias!']

https://mlco2.github.io/codecarbon/output.html#output

In [10]:
%%sh
ls

Logs
Users


In [20]:
df_emissions = pd.read_csv(os.path.join(output_dir_emissions_track,"emissions.csv"))
df_emissions.head()

,timestamp,project_name,run_id,experiment_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,2025-02-19T12:02:50,codecarbon,3bfd136c-d8e5-4f4e-a98e-a51239239023,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,17.135990,0.000041,0.000002,97.5,0.0,3.091317,...,4,Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz,NaN,NaN,-122.3303,47.6109,31.343273,process,N,1.0
1,2025-02-19T12:03:10,codecarbon,9bc68c95-44f7-4838-9b5e-00de148b7f57,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,17.254767,0.000044,0.000003,97.5,0.0,11.753727,...,4,Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz,NaN,NaN,-122.3303,47.6109,31.343273,machine,N,1.0
2,2025-02-19T12:04:54,workshop-sostenibilidad,ef6ae2a3-bd22-469f-aa2e-e12b329f5886,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,63.112533,0.000162,0.000003,97.5,0.0,11.753727,...,4,Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz,NaN,NaN,-122.3303,47.6109,31.343273,machine,N,1.0
3,2025-02-19T15:49:45,codecarbon,c29eaeff-7e73-4de2-81f5-2d7d7e121a0f,D08fa668-aed9-4ee9-a535-dedb1361039c,20.548237,0.000049,0.000002,97.5,0.0,3.047070,...,4,Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz,NaN,NaN,-122.3303,47.6109,31.343277,process,N,1.0


# Uso de la API

https://dashboard.codecarbon.io/

In [21]:
# en local
#!codecarbon login
#!codecarbon config

In [22]:

%%time
# Start tracking
tracker = EmissionsTracker(tracking_mode="process", save_to_api=True, )
tracker.start()

generate('¿Quién eres?',
         system_prompt="Eres un asistente majo",
         )
emmissions = tracker.stop()

[codecarbon INFO @ 16:05:21] Codecarbon is taking the configuration from global file: /home/azureuser/.codecarbon.config
[codecarbon DEBUG @ 16:05:21] Lock file /tmp/.codecarbon.lock already exists. This usually means another instance of codecarbon is running. You can safely delete it if you want or use allow_multiple_runs parameter to always bypass it.
[codecarbon ERROR @ 16:05:21] Error: Another instance of codecarbon is probably running as we find `/tmp/.codecarbon.lock`. Turn off the other instance to be able to run this one or use `allow_multiple_runs` or delete the file. Exiting.
[codecarbon WARNING @ 16:05:21] Another instance of codecarbon is already running. Exiting.


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
[codecarbon WARNING @ 16:05:42] Another instance of codecarbon is already running. Exiting.


CPU times: user 40.8 s, sys: 20 ms, total: 40.8 s
Wall time: 20.6 s
